# JPMorgan & Chase Co. - Quant Researcher Experience
## Task 2: Price a commodity storage contract

Suppose we have the following input parameters:
1. Injection dates 
2. Withdrawal dates
3. The prices at which the commodity can be purchased/sold on those dates
4. The volumes being injected
5. The volumes being withdrawn
6. The maximum volume that can be stored
7. Injection/Withdrawal costs
8. Storage costs
9. Transport costs

Recall that we assume that transport,buying and selling times are instantaneous, interest rates are zero and we can ignore market holidays, weekends and bank holidays.

In [189]:
import pandas as pd
import numpy as np
import datetime as datetime
from dateutil.relativedelta import relativedelta

In [190]:
NatGas = pd.read_csv('/Users/Sajeeth/Documents/Quant Experience/Task 2/Future_Nat_gas.csv')
NatGas.set_index('Dates',inplace=True)

In [191]:
# Firstly, we will adapt our function from Task 1 to find the price of natural gas at any given date between the first and last date in the dataframe
def PriceFinder(date):
    date1 = pd.to_datetime(date).date()
    date2 = pd.to_datetime(date).date() - datetime.timedelta(days=date1.day)
    date3 = date1 + relativedelta(months=1)
    date4 = date3 - datetime.timedelta(days=date3.day)
    before = NatGas.loc[date2.strftime('%Y-%m-%d')]['Prices']
    after = NatGas.loc[date4.strftime('%Y-%m-%d')]['Prices']
    price = before + ((date1.day*(after-before))/date4.day)
    return price

# Now we will define a function to find the value of the contract
def ContractValuation():
    # Input injection/withdrawal dates and volumes
    print("Enter the following parameters that will be used to calculate the value of the contract.")
    print("Write dates as YYYY-MM-DD and keep a space between each date and volume.")
    print()
    inj_dates = input("Enter your injection dates: ").split()
    inj_volumes = list(map(float, input("Enter your injection volumes (x10^6) for those corresponding dates: ").split(' ')))
    print()
    with_dates = input("Enter your withdrawal dates: ").split()
    with_volumes = list(map(float, input("Enter your withdrawal volumes (x10^6) for those corresponding dates: ").split(' ')))
    
    # Checks to ensure dates are correct and valid
    if len(inj_dates) != len(inj_volumes):
        print()
        print("The number of injection dates and corresponding injection volumes do not match")
        return
    if len(with_dates) != len(with_volumes):
        print()
        print("The number of withdrawal dates and corresponding withdrawal volumes do not match")
        return
    
    try:
        dates3 = pd.to_datetime(inj_dates).date
    except:
        print()
        print("The injection dates you have entered are in the incorrect or invalid format")
        return
    try:
        dates4 = pd.to_datetime(with_dates).date
    except:
        print()
        print("The withdrawal dates you have entered are in the incorrect or invalid format")
        return
    if ((dates3 > datetime.date(2020,10,31)) & (dates3 < datetime.date(2025,9,30))).all() == False:
        print()
        print("The injection dates are out of the specified range")
        return
    if ((dates4 > datetime.date(2020,10,30)) & (dates4 < datetime.date(2025,10,1))).all() == False:
        print()
        print("The withdrawal dates are out of the specified range")
        return
    
    # Input and verify max volume and costs are correct
    print()
    capacity = float(input("Enter the Maximum Storage Capacity (x10^6): "))
    if capacity < 0:
        print()
        print("The Maximum Storage Capacity is invalid")
        return
    print()
    cost_iw = float(input("Enter the Injection/Withdrawal Cost per 1 million cubic metres of natural gas: "))
    if cost_iw < 0:
        print()
        print("The Injection/Withdrawal Cost is invalid")
        return
    print()
    cost_storage = float(input("Enter the Storage Cost per month: "))
    if cost_storage < 0:
        print()
        print("The Storage Cost is invalid")
        return
    print()
    cost_transport = float(input("Enter the Transport Cost for each instance of transport: "))
    if cost_transport < 0:
        print()
        print("The Transport Cost is invalid")
        return
    
    # Combine all the injection and withdrawal dates and sort them in ascending order
    sorted_dates = sorted(inj_dates+with_dates, key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    # Initialise volume and revnenue
    vol = 0
    revenue = 0
    # Run a for loop over all dates and depending on if it is an injection or withdrawal date, find the new volume and revenue 
    for i in range (len(sorted_dates)):
        if sorted_dates[i] in inj_dates:
            revenue -= (PriceFinder(sorted_dates[i]) * (min(vol + inj_volumes[inj_dates.index(sorted_dates[i])],capacity) - vol) * (10**6))
            vol = min(vol + inj_volumes[inj_dates.index(sorted_dates[i])],capacity)
        else:
            revenue -= (PriceFinder(sorted_dates[i]) * (max(vol - with_volumes[with_dates.index(sorted_dates[i])],0) - vol) * (10**6))
            vol = max(vol - with_volumes[with_dates.index(sorted_dates[i])],0)
    
    # Calculate total storage costs
    storage_time = len(pd.date_range(start=sorted_dates[0], end=sorted_dates[-1], freq='M'))
    cost_storage = cost_storage * storage_time
    
    # Calculate total injection/withdrawal costs
    cost_iw = cost_iw * sum(inj_volumes+with_volumes) * len(sorted_dates)
    # Calculate total transport costs
    cost_transport = cost_transport * len(sorted_dates)
    
    # Calculate value of contract
    valuation = revenue - cost_storage - cost_iw - cost_transport
    
    print()
    print()
    # Print final value
    print('The value of the contract is', valuation, '=', format(valuation/(10**6), '.1f'), 'million approximately')
    # Print remaning volume in storage if there is any left
    if vol != 0:
        print()
        print('There is still', vol, 'x10^6 cubic metres of natural gas remaining in the storage')

In [175]:
# Test 1
ContractValuation()

Enter the following parameters that will be used to calculate the value of the contract.
Write dates as YYYY-MM-DD and keep a space between each date and volume.



Enter your injection dates:  2021-07-31 2021-06-30
Enter your injection volumes (x10^6) for those corresponding dates:  10 11


Enter your withdrawal dates:  2021-12-31 2022-01-31
Enter your withdrawal volumes (x10^6) for those corresponding dates:  10 11


Enter the Maximum Storage Capacity (x10^6):  30


Enter the Injection/Withdrawal Cost per 1 million cubic metres of natural gas:  20000


Enter the Storage Cost per month:  1000000


Enter the Transport Cost for each instance of transport:  300000




The value of the contract is 16940000.0 = 16.9 million approximately


In [176]:
# Test 2
ContractValuation()

Enter the following parameters that will be used to calculate the value of the contract.
Write dates as YYYY-MM-DD and keep a space between each date and volume.



Enter your injection dates:  2022-06-05 2023-07-12 2024-08-27
Enter your injection volumes (x10^6) for those corresponding dates:  20 15 30


Enter your withdrawal dates:  2025-01-04 2022-12-23 2023-12-16
Enter your withdrawal volumes (x10^6) for those corresponding dates:  30 10 25


Enter the Maximum Storage Capacity (x10^6):  40


Enter the Injection/Withdrawal Cost per 1 million cubic metres of natural gas:  20000


Enter the Storage Cost per month:  1000000


Enter the Transport Cost for each instance of transport:  300000




The value of the contract is 63570967.74193549 = 63.6 million approximately


In [180]:
# Test 3
ContractValuation()

Enter the following parameters that will be used to calculate the value of the contract.
Write dates as YYYY-MM-DD and keep a space between each date and volume.



Enter your injection dates:  2021-06-29 2022-07-02 2023-07-10 2024-07-07 2025-06-24
Enter your injection volumes (x10^6) for those corresponding dates:  6 14 21 15 8


Enter your withdrawal dates:  2024-12-26 2023-01-12 2022-01-04 2025-09-30 2023-12-18
Enter your withdrawal volumes (x10^6) for those corresponding dates:  11 13 4 18 18


Enter the Maximum Storage Capacity (x10^6):  100


Enter the Injection/Withdrawal Cost per 1 million cubic metres of natural gas:  20000


Enter the Storage Cost per month:  1000000


Enter the Transport Cost for each instance of transport:  300000




The value of the contract is 11651741.935483843 = 11.7 million approximately


In [184]:
# Test 4
ContractValuation()

Enter the following parameters that will be used to calculate the value of the contract.
Write dates as YYYY-MM-DD and keep a space between each date and volume.



Enter your injection dates:  2023-07-14 2021-06-27
Enter your injection volumes (x10^6) for those corresponding dates:  19 12


Enter your withdrawal dates:  2022-12-29 2024-01-12 2023-01-05
Enter your withdrawal volumes (x10^6) for those corresponding dates:  4 15 10


Enter the Maximum Storage Capacity (x10^6):  50


Enter the Injection/Withdrawal Cost per 1 million cubic metres of natural gas:  20000


Enter the Storage Cost per month:  400000


Enter the Transport Cost for each instance of transport:  300000




The value of the contract is -20414451.612903237 = -20.4 million approximately

There is still 4.0 x10^6 cubic metres of natural gas remaining in the storage
